<a href="https://colab.research.google.com/github/Shankar034/PyTorch/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PyTorch Workflow

* Collect Data (prepare and load data)
* Build Model
* fitting the model to the data(training)
* Making prediction and evaluating the model
* Saving and loading model
* Putting it all together


In [1]:
import torch
from torch import nn # nn for neural network
import matplotlib.pyplot as plt
torch.__version__

'2.0.1+cu118'


### 1. Data (Prepare and Load data)
data can be anything
* Excel Sheet
* Images
* Audio or Video
* DNA
* Text

# Machine learninig is the game of two parts:
1. get data into numerical representations
2. Building the model to learn the pattern from numerical data

In [5]:
# Known datasets
import torch
weight = 0.7
bias = 0.3
start =0
end = 1
steps=0.02
X = torch.arange(start, end, steps).unsqueeze(dim=1)
y= weight* X + bias
X[:10] ,y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [6]:
len(X), len(y)

(50, 50)

In [13]:
#Creating Training and test split
split = int(len(X)*0.8)
X_training= X[:split]
y_training= y[:split]
X_test = X[split:]
y_test=y[split:]
len(X_training), len(y_training), len(X_test), len(y_test)

(40, 40, 10, 10)